# Automatically count cars from a video streaming using morphological transformations and OpenCV.

This is a basis introduction for moving object detection over a video streaming. We detect contours (cars) by computing diference between two frames (images) from the incoming video and keep track of such objects, also known as Blobs.

Once those blobs are detected, we increase a counter when a line (customly drawn on the video) is met by the blob.

### Import the required libraries.

In [3]:
import cv2
import numpy as np
from time import sleep

### Set parameters

In [5]:
#min width and heigth (in pixels) of a blob to be considered a car.
width_min=50
heigth_min=50 

#Max error threshold between blobs (distance from the same blob between 2 frames to consider it as the same one).
offset=6 

#Position (in pixels) over the Y axis where the like is drawn.
line_pos=550 #Posição da linha de contagem 

#Max video FPS
delay= 60

#List to keep track of the active blobs on the video.
detected = []

#counter
cars = 0

### Iterate over video.

In [6]:
#set input video
cap = cv2.VideoCapture('./data/video.mp4')
#Initialize background segmentator (To extract the moving foreground from static background).
sub = cv2.bgsegm.createBackgroundSubtractorMOG()

#Get center pixels of a given box.
def get_center(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx,cy

#Iterate video frames
while True:
    #Read next frame
    ret , frame1 = cap.read()
    #FPS
    time = float(1/delay)
    sleep(time)
    #Convert image to grayscale
    grey = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    #Add blur to image in order to increase performance on blob detection
    blur = cv2.GaussianBlur(grey,(3,3),5)
    #Get the moving foreground from video (vehicles)
    img_sub = sub.apply(blur)
    #Apply morphological operations to image.
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilat2 = cv2.morphologyEx (dilat, cv2. MORPH_CLOSE , kernel)
    dilat2 = cv2.morphologyEx (dilat2, cv2. MORPH_CLOSE , kernel)
    #Get blobs
    blobs,h=cv2.findContours(dilat2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    #Draw line
    cv2.line(frame1, (25, line_pos), (1200, line_pos), (255,127,0), 3) 
    for(i,c) in enumerate(blobs):
        #Get the bounding box for the blob
        (x,y,w,h) = cv2.boundingRect(c)
        #Validate min width and heigth to be considered as a possible car
        if not ((w >= width_min) and (h >= heigth_min)):
            continue
        #Draw bounding box on moving blob
        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)    
        #Get blob's center
        center = get_center(x, y, w, h)
        #Add blob's center to detected cars
        detected.append(center)
        #Draw center of the blob.
        cv2.circle(frame1, center, 4, (0, 0,255), -1)

        #Validate if any of the detected blobs has already met the threshold
        for (x,y) in detected:
            if y<(line_pos+offset) and y>(line_pos-offset):
                cars+=1
                #Change color line to denote a car has been counted.
                cv2.line(frame1, (25, line_pos), (1200, line_pos), (0,127,255), 3)  
                #Remove already counted car from list.
                detected.remove((x,y))
                print("Total cars detected : "+str(cars))        
    #Show the counter after the whole frame has been analized   
    cv2.putText(frame1, "Total cars : "+str(cars), (450, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255),5)
    cv2.imshow("Original video" , frame1)
    cv2.imshow("Blobs",dilat2)

    #Exit program by pressing "esc" key
    if cv2.waitKey(1) == 27:
        break

#Free resources
cv2.destroyAllWindows()
cap.release()

Total cars detected : 1
Total cars detected : 2
Total cars detected : 3
Total cars detected : 4
Total cars detected : 5
Total cars detected : 6
Total cars detected : 7
Total cars detected : 8
Total cars detected : 9
Total cars detected : 10
Total cars detected : 11
Total cars detected : 12
Total cars detected : 13
Total cars detected : 14
Total cars detected : 15
Total cars detected : 16
Total cars detected : 17
Total cars detected : 18
Total cars detected : 19
Total cars detected : 20
Total cars detected : 21
Total cars detected : 22
Total cars detected : 23
Total cars detected : 24
Total cars detected : 25
Total cars detected : 26
Total cars detected : 27
Total cars detected : 28
Total cars detected : 29
Total cars detected : 30
Total cars detected : 31
Total cars detected : 32
